Putting some old code in here, need to separate out the visualizing stuff to put into the vis file. WIP

In [ ]:
#terminal installs if needed
#!sudo apt-get install python3-dev graphviz libgraphviz-dev pkg-config
# !pip install pygraphviz
# !pip install datasets;
# !pip install diffusers


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.spatial.distance import hamming
import networkx as nx
import torch
import torchvision
from torch import nn
from torchvision.transforms  import Compose
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import copy
from diffusers import DDPMScheduler, UNet2DModel, UNet2DConditionModel
import pickle

import pygraphviz as pgv
import numba
from numba.experimental import jitclass

In [ ]:
#specs = [('n', numba.int32), ('b', numba.float64[:]), ('W', numba.float64[:]), ('s', numba.float64[:])]

#@jitclass(specs)
class hopfield:
  def __init__(self, setn):
    self.n = setn
    self.b = np.random.rand(setn) * 2 - 1
    w = np.zeros((setn, setn))
    for i in range(setn):
      for j in range(i, setn):
        w[i, j] = w[j,i] = np.random.rand()
    self.W = (w * 2 - 1)
    self.s = np.random.choice([-1,1], (setn))

  def E(self):
    sum1 = 0
    sum2 = 0
    for i in range(self.n):
      sum2 += self.s[i] * self.b[i]
      for j in range(i + 1, self.n):
        sum1 += self.s[i] * self.s[j] * self.W[i, j]
    return -0.5 * sum1 + sum2

  def _newstate(self, j):
    sum = 0
    for i in range(self.n):
      if (i != j):
        sum += self.s[i] * self.W[i,j]
    return 1 if sum > self.b[j] else -1

  def step(self):
    j = np.random.randint(self.n)
    self.s[j] = self._newstate(j)

  def showImage(self, title=None):
    im = np.copy(self.s)
    im2 = im.reshape(int(self.n**0.5), int(self.n**0.5))
    plt.matshow(im2)
    if title is not None:
      plt.title(title)
    plt.show()

  def equilibrium(self):
    for j in range(self.n):
      if self._newstate(j) != self.s[j]:
        return False
    return True

  def getIntState(self):
    binstate = self.s * 0.5 + 0.5
    return int("".join(str(int(x)) for x in binstate), 2)

  def setIntState(self, i):
    self.s = np.array([int(c) for c in format(i, '0' + str(self.n) + 'b')]) * 2 - 1


In [ ]:
def sweep(hop):
  for i in range(hop.n):
    hop.step()
  return hop.E()

In [ ]:
for j in range(10):
  sweeps = 11
  h = hopfield(100)
  init = h.E()
  E = [sweep(h) for i in range(sweeps)]
  plt.plot(list(range(sweeps+1)), [init] + E)
  plt.ylabel("Energy")
  plt.xlabel("Sweeps (100 steps)")
  plt.title("Energy vs Sweeps for 10 n=100 hopfield networks")
plt.show()

In [ ]:
face = '0000000000000100010000000000000000000000000010000000000000000001110000001000100001000001101000000001'
tree = '0001111000000111100000001100000000110000001111111000001100100000110000000011000000001100000000110000'
facevec = [-1 if i == '0' else 1 for i in face]
treevec = [-1 if i == '0' else 1 for i in tree]
arr = np.array(facevec).reshape(int(len(facevec)**0.5), int(len(facevec)**0.5))
plt.matshow(arr)

In [ ]:
def learnimages(imvecs):
  sum = np.outer(imvecs[0], imvecs[0])
  for vec in imvecs[1:]:
    sum += np.outer(vec, vec)
  w = sum / len(imvecs)
  h = hopfield(len(imvecs[0]))
  h.W = w
  return h

In [ ]:
def perturbRestore(hop, im, sweeps):
  imvec = [-1 if i == '0' else 1 for i in im]
  # perturbs = np.random.choice(len(imvec), int(len(imvec)*percentperturb), replace=False)
  arr = np.array(imvec).reshape(int(len(imvec)**0.5), int(len(imvec)**0.5))
  for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
      if j <= math.ceil(arr.shape[0] * 0.4):
        arr[i,j] = 1
  hop.s = arr.flatten()
  hop.showImage("Original Corruption")
  print(hop.W.shape, hop.s.shape)
  for sweep in range(sweeps):
    for i in range(hop.n):
      hop.step()
    hop.showImage(f"After {sweep + 1} sweeps")

In [ ]:
hopnet = learnimages([facevec, treevec])
perturbRestore(hopnet, face, 10)

In [ ]:
def kbitsperturbed(imgvec, k):
  copyimgvec = np.copy(imgvec)
  perturbs = np.random.choice(len(imgvec), k, False)
  for p in perturbs:
    copyimgvec[p] *= -1
  return copyimgvec

In [ ]:
#@numba.njit
def nsteps(hopf, n):
  for i in range(n):
    hopf.step()
  return hopf

In [ ]:
mem = np.random.choice([1,-1], 100)
print(mem)
pmem = kbitsperturbed(mem, 30)
print(mem)
print(pmem)
hamming(mem, pmem) * len(mem)

In [ ]:
memcounts = 70
n = 100
kcounts = 70
scores = np.zeros((memcounts, kcounts))
for memcount in range(1, memcounts):
  for memiters in range(1):
    memlist = [np.random.choice([1,-1], n) for i in range(memcount)]
    hopf = learnimages(memlist)
    for k in range(kcounts):
      for setiters in range(10):
        memorytouse = memlist[np.random.randint(memcount)]
        perturbmem = kbitsperturbed(memorytouse, k)
        hopf.s = perturbmem
        #print(hopf.s.shape, hopf.W.shape)
        while (not hopf.equilibrium()):
          hopf = nsteps(hopf, n)

        thishamming = hamming(hopf.s, memorytouse) * n
        scores[memcount, k] += thishamming
averagescores = scores / 10


In [ ]:
averagescores *= 10
print(averagescores)
plt.matshow(averagescores)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot()
cax = ax.matshow(averagescores.T) # Transpose because axes were being displayed as flipped
fig.colorbar(cax)
plt.xlabel("Number of Memories")
plt.ylabel("Percentage Perturbed")
plt.title("Hamming Distance from a randomly chosen image\nfrom trained hopfield after perturbation")
plt.show()

In [ ]:
h4 = hopfield(4)
h4.setIntState(13)
print(h4.s)
h4.getIntState()

n = 6
memcount = 2
h6 = hopfield(n)
memlist = [np.random.choice([1,-1], n) for i in range(memcount)]
dg = nx.DiGraph()
dg.add_nodes_from(range(2**n))
for mem in memlist:
  h6.s = mem
  state = h6.getIntState()
  dg.nodes[state]['style'] = 'filled'
  dg.nodes[state]['fillcolor'] = 'red'


h6 = learnimages(memlist)
for i in range(2**n):
  h6.setIntState(i)
  for j in range(n):
    h6.s[j] = h6._newstate(j)
    newState = h6.getIntState()
    if newState != i:
      dg.add_edge(i, newState)
    h6.setIntState(i)

In [ ]:
#https://stackoverflow.com/questions/39657395/how-to-draw-properly-networkx-graphs
A = nx.drawing.nx_agraph.to_agraph(dg)
A.layout('dot')
A.draw('vis.png')